In [1]:
import earthaccess
fs = earthaccess.get_fsspec_https_session()

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import earthaccess
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd  
import os 
import sys
sys.path.append("/work")
from pathlib import Path
from helper import *
from dask.distributed import progress

In [2]:
from dask.distributed import Client
client = Client()

2025-10-25 14:38:17,413 - distributed.scheduler - WARNING - Failed to format dashboard link, unknown value: 'JUPYTERHUB_SERVICE_PREFIX'


2025-10-25 14:39:57,049 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:37287' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'grid_match-af85df3640a042a95315e60068a07b07', 'grid_match-35cc51dfffd6b421e98229848ee6143e', 'grid_match-1e70a59b0f21bdb150bf73e099dea471', 'grid_match-9249d56cf31cf91ef817d140f90c2c02', 'grid_match-6f93625cd70c5cfedda9fbdf4771a223', 'grid_match-046bcbe4a5190c8be7668e579f3ab2bc', 'grid_match-56bfcc6625dcc159ae654ba8e631997d', 'grid_match-14010ccdb7c102c66b749809b014db2e', 'grid_match-e888bf4a502739cfbbaef10e986f29f6', 'grid_match-1228daa5696e516e3cc892f5c61a924f', 'grid_match-574ebb1aa4a96862d8dee2fd00d28e6c', 'grid_match-0747654eb8a2a0b9d25e808656ba8bc7', 'grid_match-8b2d5efd5098d69cba1edd6a46f39f0f', 'grid_match-b6b54f56184653895b3e5cf97d24bd60', 'grid_match-9d00d9ed3ea2a3d4bd5127a7d737e49c', 'grid_match-fcb802d717da17d681e289bb67c7bed8', 'grid_match-b2b56064263e6ee5b89d28c609e07485', 'grid_match

In [3]:
data_dir = Path("/work/chla_NAO/")
interp_dir = Path("/work/chla_NAO/")

# NOA CHLA
region=(-88.242188,4.214943,-2.988281,44.465151)
tspan = ("2024-09-01 00:00", "2024-11-01 00:00")

# PATHS
paths_oci_all = list(data_dir.glob(f"oci/*"))
paths_spx_all = list(data_dir.glob(f"spexone/*"))


In [4]:
dt_oci = xr.open_datatree(paths_oci_all[0])
# geophysical data/chlor_a
dt_spx = xr.open_datatree(paths_spx_all[0])
# geophysical data/chla

In [5]:
batch = 0
batch_size = 200

In [6]:
start_ind = batch * batch_size
end_ind = min((batch  + 1 )* batch_size, len(paths_oci_all))

In [9]:
output_path = interp_dir / f"oci_L3M/oci_{batch_size}_{batch}.nc"
if os.path.exists(output_path):
    print("Loading cached data from file...")
    da_oci = xr.load_dataarray(output_path)
else:
    if start_ind >= end_ind:
        print("start_ind >= end_ind")
    else:
        paths_oci = paths_oci_all[start_ind : end_ind]

        crs, shape_tmp, transform_tmp = crs_template(paths_oci[0], "chlor_a")
        shape, transform, _ = grid_aligned_subset(region, transform_tmp, shape_tmp)

        kwargs = {"combine": "nested", "concat_dim": "time"}
        attrs = xr.open_mfdataset(paths_oci, preprocess=time_from_attr, **kwargs)

        
        print("Computing data...")

        futures = client.map(
            grid_match,
            paths_oci,
            dst_crs=crs,
            dst_shape=shape,
            dst_transform=transform,
            var="chlor_a",
        )
        progress(futures)  # <---- displays progress bar inline (Jupyter or console)
        da_oci = xr.combine_nested(client.gather(futures), concat_dim="time")
        da_oci["time"] = attrs["time"]
        da_oci = da_oci['chlor_a']
        da_oci.to_netcdf(output_path)
        print(f"Saved to {output_path}")

KeyboardInterrupt: 

Exception ignored in: 'rasterio._env.log_error'
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.11/logging/__init__.py", line 1479, in info
    def info(self, msg, *args, **kwargs):

KeyboardInterrupt: 


Computing data...


KeyboardInterrupt: 

In [ ]:
output_path = interp_dir / f"spexone_L3M/spexone_{batch_size}_{batch}.nc"
if os.path.exists(output_path):
    print("Loading cached data from file...")
    da_spx = xr.load_dataarray(output_path)
else:
    if start_ind >= end_ind:
        print("start_ind >= end_ind")
    else:
        print("Computing data...")
        
        start_ind = batch * batch_size
        end_ind = min((batch  + 1 )* batch_size, len(paths_spx_all))
        paths_spx = paths_spx_all[start_ind : end_ind]

        crs, shape_tmp, transform_tmp = crs_template(paths_spx[0], "chla")
        shape, transform, _ = grid_aligned_subset(region, transform_tmp, shape_tmp)
        
        kwargs = {"combine": "nested", "concat_dim": "time"}
        attrs = xr.open_mfdataset(paths_spx, preprocess=time_from_attr, **kwargs)
        futures = client.map(
            grid_match,
            paths_spx,
            dst_crs=crs,
            dst_shape=shape,
            dst_transform=transform,
            var="chla",
        )
        progress(futures)  # <---- displays progress bar inline (Jupyter or console)
        da_spx = xr.combine_nested(client.gather(futures), concat_dim="time")
        da_spx["time"] = attrs["time"]
        da_spx = da_spx['chla']
        da_spx.to_netcdf(output_path)
        print(f"Saved to {output_path}")

In [ ]:
da_spx_filter = da_spx.where(da_spx != 0)  # keep NaN for filtered-out zeros

# Keep only times where at least one lon/lat cell has a real value
da_spx_filter = da_spx_filter.where(
    da_spx_filter.count(dim=["latitude", "longitude"]) > 0,
    drop=True
)

In [ ]:
da_oci_spx = da_oci.interp_like(da_spx_filter)

In [ ]:
da_oci_spx.mean(dim='time').plot()